# Install Necessary Packages (Only need to do once)

In [1]:
# Install Necessary Packages
! pip install geopandas
! pip install shapely
! pip install opencv-python


In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
# Importing the toolbox (takes several seconds)
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd

# Import modules
CUSTOM_UTILS_PATH = Path(os.getcwd()).parent/'src'/'behavior'/'pose_estimation'
if CUSTOM_UTILS_PATH.exists():
    sys.path.append(str(CUSTOM_UTILS_PATH))
    # Import custom analysis modules
    try:
        import dlc_utils
        import grid_utils
        import notebook_utils
        print("Labyrinth Utils loaded successfully!")
    except ImportError as e:
        print(f"Warning: Could not import custom modules: {e}")
else:
    print(f"Warning: Custom utils path not found: {CUSTOM_UTILS_PATH}")

Location of Computer 1 Videos: 
Location of Computer 2 Videos: 
Central Video Location: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\videos\original_videos
DLC Scorer: DLC_resnet50_LabyrinthMar13shuffle1_1000000
Tracking bodyparts: nose, belly, sternum, leftflank, rightflank, tailbase
Experimental groups: ['A', 'B', 'C', 'D']
Labyrinth Utils loaded successfully!


c:\Users\PalopLabPortal\Anaconda\envs\compass-labyrinth\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


### Specify Paths and Other Information

In [4]:
# =============================================================================
# CREATE PROJECT DIRECTORY AND SUBFOLDERS FROM CONFIG FILE
# =============================================================================
from init_config import *

print()
print(f'base_path: {BASE_PATH}')
DIRS = notebook_utils.create_organized_directory_structure(BASE_PATH)


base_path: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS
Creating organized directory structure...
videos: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\videos
videos_original: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\videos\original_videos
frames: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\videos\frames
data: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\data
dlc_results: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\data\dlc_results
dlc_cropping: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera\Labyrinth Mazes discussions\Code\TEST_COMPASS\data\dlc_cropping_bounds
grid_files: c:\Users\PalopLabPortal\Gladstone Dropbox\Shreya Bangera

In [32]:
# Set some plotting parameeters
# Load in the Spike and LFP Data
from pylab import *
%matplotlib inline
rcParams['figure.figsize']=(12,3)  # Change the default figure size

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['svg.fonttype'] = 'none'

# Import Mouse Metadata

In [33]:
# Load the metadata file
now = datetime.datetime.now(); print(now)

# Add Metadata to metadata directory
mouseinfo = dlc_utils.load_cohort_metadata(METADATA_FILE, TRIAL_TYPE)

# Validate the metadata
dlc_utils.validate_metadata(mouseinfo)

# Display summary of the metadata
dlc_utils.display_metadata_summary(mouseinfo)

mouseinfo

2025-08-15 15:17:22.362418
Loading metadata from: c:\Users\patrick.honma\TEST_COMPASS\data\metadata\20241028_WT_DSI_Labyrinth_DLC_InfoSheet_v1.xlsx
Sheet name: Labyrinth_DSI
Initial rows loaded: 12
Excluded 0 trials marked for exclusion
Final dataset: 5 trials
Metadata validation completed

METADATA SUMMARY
Total trials: 5
Columns: ['Session #', 'Run Date', 'Time Of Day', 'Rack Location', 'Computer', 'Noldus Chamber', 'Camera #', 'Noldus Trial', 'Noldus Project Name', 'DSI Trial', 'ID', 'Name', 'Sex', 'Status', 'Line', 'Genotype', 'GenotypeOG', 'Birth Date', 'Age (months)', 'Housing ID', 'Transmitter #', 'Exclude Trial', 'X1', 'X2', 'Y1', 'Y2', 'NOTES']
Session # range: 3 - 7
Missing data:


,Session #,Run Date,Time Of Day,Rack Location,Computer,Noldus Chamber,Camera #,Noldus Trial,Noldus Project Name,DSI Trial,...,Birth Date,Age (months),Housing ID,Transmitter #,Exclude Trial,X1,X2,Y1,Y2,NOTES
0,3,2024-11-18,17:15:00,BR,1,TR,6,1,20241118_WT_DSI_Labyrinth_2,2,...,2024-06-01,5m13d,H-11495-17,"3,4",NaN,1486,2399,70,986,Grabbed all pellets early on
1,4,2024-11-18,17:15:00,TR,1,BR,2,1,20241118_WT_DSI_Labyrinth_2,2,...,2024-06-02,5m12d,H-11497-17,"5,6",NaN,1435,2359,1086,2015,Light turned off; slightly dark
2,5,2024-11-20,17:15:00,BR,1,TR,4,3,20241118_WT_DSI_Labyrinth_2,4,...,2024-06-01,5m13d,H-11495-17,"3,4",NaN,1486,2399,70,986,Entered
3,6,2024-12-02,17:15:00,BR,1,TR,4,5,20241118_WT_DSI_Labyrinth_2,6,...,2024-06-02,5m12d,H-11497-17,"3,4",NaN,1486,2399,70,986,Entered
4,7,2024-12-03,17:15:00,BL,1,TL,1,6,20241118_WT_DSI_Labyrinth_2,7,...,2024-06-01,5m13d,H-11495-17,"1,2",NaN,144,1075,19,954,Entered


# OPTIONAL: Copy and rename videos from original location to VIDEOFILE_PATH


In [ ]:
now = datetime.datetime.now(); print(now)

# 1. Save first frames
copy_results = dlc_utils.copy_and_rename_videos(
            mouseinfo_df=mouseinfo,
            video_paths=[VIDEO_PATH_1, VIDEO_PATH_2], 
            destination_path=DIRS['videos_original'],
        )

# Labyrinth_DLC_Utils.print_copy_summary(copy_results)

2025-08-15 15:21:01.056757
Error: No valid video source paths found!


# Get DeepLabCut Cropping Bounds

In [ ]:
# Save first frames for all videos
now = datetime.datetime.now(); print(datetime.datetime.now())

print("\nSaving first frames for all videos...")
frame_results = dlc_utils.batch_save_first_frames(
    mouseinfo_df=mouseinfo,
    video_directory=DIRS['videos_original'],
    frames_directory=DIRS['frames']
)

2025-08-15 15:22:16.912854

Saving first frames for all videos...
Saving first frames for 5 sessions...
Video directory: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos
Frames directory: c:\Users\patrick.honma\TEST_COMPASS\videos\frames

Processing Session-3...
  Video not found: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos\Session-3.mp4

Processing Session-4...
  Video not found: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos\Session-4.mp4

Processing Session-5...
  Video not found: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos\Session-5.mp4

Processing Session-6...
  Video not found: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos\Session-6.mp4

Processing Session-7...
  Video not found: c:\Users\patrick.honma\TEST_COMPASS\videos\original_videos\Session-7.mp4


In [43]:
# Get the DLC cropping bounds for a all videos
now = datetime.datetime.now(); print(datetime.datetime.now())

coordinates_dict = dlc_utils.batch_get_boundary_and_cropping(
    mouseinfo_df=mouseinfo, 
    frames_directory=DIRS['frames'],
    cropping_directory=DIRS['dlc_cropping'],
    boundaries_directory=DIRS['grid_boundaries']
    )

2025-08-15 15:22:30.338499
Getting boundary points and cropping coordinates for 5 sessions...
This replaces both DLC cropping selection and boundary selection!
Press 'c' to skip a session, or ESC to stop completely.

Processing Session-3 (1/5)
Chamber: TR
Error: Frame not found at c:\Users\patrick.honma\TEST_COMPASS\videos\frames\Session-3Frame1.jpg
Run batch_save_first_frames() first to create the frame.
✗ Skipped Session-3

Processing Session-4 (2/5)
Chamber: BR
Error: Frame not found at c:\Users\patrick.honma\TEST_COMPASS\videos\frames\Session-4Frame1.jpg
Run batch_save_first_frames() first to create the frame.
✗ Skipped Session-4

Processing Session-5 (3/5)
Chamber: TR
Error: Frame not found at c:\Users\patrick.honma\TEST_COMPASS\videos\frames\Session-5Frame1.jpg
Run batch_save_first_frames() first to create the frame.
✗ Skipped Session-5

Processing Session-6 (4/5)
Chamber: TR
Error: Frame not found at c:\Users\patrick.honma\TEST_COMPASS\videos\frames\Session-6Frame1.jpg
Run batch

In [ ]:
# Get the DLC cropping bounds for a single video
coordinates_dict = dlc_utils.get_labyrinth_boundary_and_cropping(
    frames_directory=DIRS['frames'],
    cropping_directory=DIRS['dlc_cropping'],
    boundaries_directory=DIRS['grid_boundaries'],
    session='Session0007',
    chamber_info=None)


Labyrinth Boundary Selection for Session-7
Image size: 2560 x 2048 (W x H)

Instructions:
1. Click on the 4 corners in this order:
   - Top-Left corner
   - Bottom-Left corner
   - Bottom-Right corner
   - Top-Right corner
2. Right-click to see pixel RGB values (optional)
3. Press 'q' to confirm selection after clicking 4 corners
4. Press 'r' to reset and select again
5. Press 'c' to cancel
Top-Left corner: (177, 51)
Click Bottom-Left corner next...
Bottom-Left corner: (177, 932)
Click Bottom-Right corner next...
Bottom-Right corner: (1058, 927)
Click Top-Right corner next...
Top-Right corner: (1052, 44)
4 corners selected. Cropping bounds: X1=177, X2=1058, Y1=44, Y2=932
Press 'q' to confirm, 'r' to reset

Results for Session-7:
Boundary corners:
  Top-Left: (177, 51)
  Bottom-Left: (177, 932)
  Bottom-Right: (1058, 927)
  Top-Right: (1052, 44)
Derived cropping bounds: X1=177, X2=1058, Y1=44, Y2=932
Cropping size: 881 x 888 pixels
Boundary points saved to: C:\Users\patrick.honma\Glads

# Analyze the Videos with DeepLabCut

In [ ]:
now = datetime.datetime.now(); print(datetime.datetime.now())

analysis_results = dlc_utils.analyze_videos_with_DLC(
        mouseinfo_df=mouseinfo,
        config_path=DLC_CONFIG_PATH,
        video_directory=DIRS['videos_original'],
        cropping_directory=DIRS['dlc_cropping'],
        results_directory=DIRS['dlc_results']
    )

2025-06-25 14:33:34.290326
DeepLabCut analysis started: 2025-06-25 14:33:34.291323
Video directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\videos\original_videos
Cropping directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_cropping_bounds
Results directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_results
-----------------------------
Analyzing Session-3...
Video: Session-3.mp4
Using saved cropping bounds: (1493, 2385, 92, 980)
Analysis already exists for Session-3, skipping...
-----------------------------
Analyzing Session-4...
Video: Session-4.mp4
Using saved cropping bounds: (1449, 2353, 1132, 2040)
Analysis already exists for Session-4, skipping...
-----------------------------
Analyzing Session-5...
Video: Session-5.mp4
Using saved cropping bounds: (1491, 2385, 87, 983)
Anal

# Create Grids and save as Grid Files

In [ ]:
now = datetime.datetime.now()

print(f"\nCreating grids for {len(mouseinfo)} sessions...")

# Run batch grid creation
grid_results = dlc_utils.batch_create_grids(
    mouseinfo_df=mouseinfo,
    boundaries_directory=DIRS['grid_boundaries'],
    grid_files_directory=DIRS['grid_files'],
    cropping_directory=DIRS['dlc_cropping'],
    num_squares=12
)


Creating grids for 5 sessions...
Batch grid creation started: 2025-06-25 15:19:36.916668
Grid size: 12 x 12
Creating grids for 5 sessions...
Boundary points directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\grid_boundaries
Cropping coordinates directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_cropping_bounds
Grid files directory: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\grid_files
Grid coordinates will be adjusted to cropped frame coordinate system
-----------------------------
Processing Session-3 (1/5)...
Chamber: TR
Session-3 grid already exists!
-----------------------------
Processing Session-4 (2/5)...
Chamber: BR
Session-4 grid already exists!
-----------------------------
Processing Session-5 (3/5)...
Chamber: TR
Session-5 grid already exists!
-------------------------

c:\Users\patrick.honma\AppData\Local\anaconda3\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


# Initial Visualizations

### Plot the Scatterplot with Grid overlayed for each trial

In [ ]:
now = datetime.datetime.now()

print("\n--- Batch Processing Example ---")
batch_results = dlc_utils.batch_create_grid_scatter_plots(
    mouseinfo_df=mouseinfo,
    dlc_results_directory=DIRS['dlc_results'],
    grid_files_directory=DIRS['grid_files'],
    figures_directory=DIRS['figures'],
    dlc_scorer=DLC_SCORER,
    bodypart='sternum',
    likelihood_threshold=0.6,
    figure_size=(3, 3),
    show_plots=False  # Don't display plots during batch processing
)


--- Batch Processing Example ---
Batch creating grid scatter plots started: 2025-06-25 15:34:07.510309
Creating scatter plots for 5 sessions...
Bodypart: sternum
Likelihood threshold: 0.6
Saving plots to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\figures\scatter_plots
-----------------------------
Creating grid scatter plot for Session-3...
  Loaded grid: 144 grid squares
  Loaded DLC data: 234064 frames
  Plotted 70845/234064 points above likelihood threshold
  Plot saved to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\figures\scatter_plots\Session-3_sternum_scatter_plot.png
✓ Plot created for Session-3
-----------------------------
Creating grid scatter plot for Session-4...
  Loaded grid: 144 grid squares
  Loaded DLC data: 234064 frames
  Plotted 222232/234064 points above likelihood threshold
  Plot saved to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\

### Create Trajectory Plots with Grid Overlaid

In [ ]:
now = datetime.datetime.now()

print("\n--- Different Colormaps Example ---")
colormaps = ['viridis']

for colormap in colormaps:
    print(f"\nCreating trajectory plots with {colormap} colormap...")
    batch_results = dlc_utils.batch_create_trajectory_plots(
        mouseinfo_df=mouseinfo,  # Just first 2 sessions
        dlc_results_directory=DIRS['dlc_results'],
        grid_files_directory=DIRS['grid_files'],
        figures_directory=DIRS['figures'] / f"trajectory_plots",
        dlc_scorer=DLC_SCORER,
        bodypart='sternum',
        likelihood_threshold=0.6,
        colormap=colormap,
        show_plots=True
    )

# Create CSVs with Grid Numbers

In [ ]:
now = datetime.datetime.now()

results = dlc_utils.batch_append_grid_numbers(
    mouseinfo_df=mouseinfo,
    grid_files_directory=DIRS['grid_files'],
    dlc_results_directory=DIRS['dlc_results'],
    dlc_scorer=DLC_SCORER,
    save_directory=DIRS['dlc_results']
)

Batch grid annotation started: 2025-06-25 15:59:30.540184
Annotating 5 sessions with grid numbers...
-----------------------------
Processing Session-3 (1/5)...
Appending grids to Session-3 CSV...
Saved to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_results\Session-3_withGrids.csv
Grid annotation completed for Session-3
-----------------------------
Processing Session-4 (2/5)...
Appending grids to Session-4 CSV...
Saved to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_results\Session-4_withGrids.csv
Grid annotation completed for Session-4
-----------------------------
Processing Session-5 (3/5)...
Appending grids to Session-5 CSV...
Saved to: C:\Users\patrick.honma\Gladstone Dropbox\Palop Lab\Patrick\DeepLabCut Projects\WT_DSI_Labyrinth_TESTING\data\dlc_results\Session-5_withGrids.csv
Grid annotation completed for Session-5
--------------------------